In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

from jcamp import jcamp_readfile, jcamp_calc_xsec
from rdkit import Chem
from scipy.interpolate import interp1d
from tqdm import tqdm

In [2]:
PATH = "jdx"
SMILES = []
spectra = []

In [3]:
def extremum(JDX_PATH):
    """
    Find extremum in JDX file:
    * _x_begin_max is the maximum value at the left end of the X-axis of all spectral data
    * _x_end_min is the minimum value at the right end of the X-axis for all spectral data
    """
    print('Looking for extremum in JDX file...')
    _x_begin_max, _x_end_min, sign = None, None, False
    for file in tqdm(os.listdir(JDX_PATH)):
        try:
            _spectrum = jcamp_readfile(JDX_PATH + "/" + file)
            try:
                if _spectrum["state"] != "gas" and _spectrum['state'] != "GAS":
                    continue
                jcamp_calc_xsec(_spectrum, skip_nonquant=False)
            except:
                continue

            x = _spectrum['wavenumbers']
            if _spectrum['xunits'].lower() in ('1/cm', 'cm-1', 'cm^-1'):
                pass
            elif _spectrum['xunits'].lower() in ('micrometers', 'um', 'wavelength (um)'):
                x = x[::-1]
            elif _spectrum['xunits'].lower() in ('nanometers', 'nm', 'wavelength (nm)'):
                x = x[::-1]

            if _spectrum['yunits'].lower() == 'transmittance':
                y = _spectrum['absorbance']
            elif _spectrum['yunits'].lower() == 'absorbance':
                y = _spectrum['y']
            elif _spectrum['yunits'].lower() == '(micromol/mol)-1m-1 (base 10)':
                continue
        except:
            continue

        if type(y) != np.ndarray:
            continue

        _mol_file = "mol/" + file[:-7] + ".mol"
        _mol = Chem.MolFromMolFile(_mol_file)
        try:
            _sm = Chem.MolToSmiles(_mol)
        except:
            continue

        if _sm != "":
            if not sign:
                sign = True
                _x_begin_max = x[0]
                _x_end_min = x[-1]
            else:
                _x_begin_max = _x_begin_max if _x_begin_max > x[0] else x[0]
                _x_end_min = _x_end_min if _x_end_min < x[-1] else x[-1]
    
    print('_x_begin_max: ' + str(_x_begin_max) + ', _x_end_min: ' + str(_x_end_min))
    
    return _x_begin_max, _x_end_min

In [4]:
def closest(myList, myNumber):
    """
    Find num in list closest to mynum, used in preprocess.
    """
    from bisect import bisect_left
    pos = bisect_left(myList, myNumber)
    if pos == 0:
        return myList[0]
    if pos == len(myList):
        return myList[-1]
    before = myList[pos - 1]
    after = myList[pos]
    if after - myNumber < myNumber - before:
       return after
    else:
       return before

def process_file(filename, begin, end):
    """
    Convert spectrum units to wavenumbers (x) and absorbance (y), and interpolate any missing coordinates, then return a list of absorbance.
    """
    # TODO: ask if nitrogen dilution matters at all
    # MIN: begin, MAX: end, increments of 2
    spectrum = jcamp_readfile(filename)
    try:
        if spectrum["state"] != "gas" and spectrum['state'] != "GAS":
            return False
        jcamp_calc_xsec(spectrum, skip_nonquant = False)
    except:
        return False
    
    x = spectrum['wavenumbers']
    if spectrum['xunits'].lower() in ('1/cm', 'cm-1', 'cm^-1'):
        pass
    elif spectrum['xunits'].lower() in ('micrometers', 'um', 'wavelength (um)'):
        x = x[::-1]
    elif spectrum['xunits'].lower() in ('nanometers', 'nm', 'wavelength (nm)'):
        x = x[::-1]
    
    if spectrum['yunits'].lower() == 'transmittance':
        y = spectrum['absorbance']
    elif spectrum['yunits'].lower() == 'absorbance':
        y = spectrum['y']
    elif spectrum['yunits'].lower() == '(micromol/mol)-1m-1 (base 10)':
        return
    
    # Define the desired wave number range
    x_new = np.arange(begin, end+1, 2)

    # Interpolate the absorbance values
    f = interp1d(x, y, kind='slinear', bounds_error=False, fill_value="extrapolate")
    y_new = f(x_new)

    return y_new
    # plt.plot()

def why_bad(filename):
    spectrum = jcamp_readfile(filename)
    try:
        jcamp_calc_xsec(spectrum, skip_nonquant = False)
        if spectrum["state"] != "gas" and spectrum['state'] != "GAS":
            return spectrum["state"]
    except:
        return "Couldn't read file"
    pass

In [5]:
bad, reason_bad = 0, []
x_begin_max, x_end_min = extremum(PATH)
print("------ ------ ------ ------ ------ ------")
print("Preprocessing spectrum...")
for spectra_file in tqdm(os.listdir(PATH)):
    try:
         absorbance = process_file(PATH + "/" + spectra_file, x_begin_max, x_end_min)
    except:
        bad += 1
        continue
    
    if type(absorbance) != np.ndarray:
        reason_bad.append(why_bad(PATH + "/" + spectra_file))
        continue
    
    # Get the smiles out of the mol using rdkit
    mol_file = "mol/" + spectra_file[:-7] + ".mol"
    mol = Chem.MolFromMolFile(mol_file)
    try:
        sm = Chem.MolToSmiles(mol)
    except:
        continue  
    
    if sm != "":
        """
        spectra: Save all correctly processed spectral data
        SMILES: Save the molecular structure of the spectrum
        """
        spectra.append(absorbance)
        SMILES.append(sm)

spectra = np.array(spectra)

Looking for extremum in JDX file...


 28%|██▊       | 4471/16121 [01:06<02:52, 67.55it/s] [23:00:23] Warning: ambiguous stereochemistry - opposing bonds have opposite wedging - at atom 6 ignored.
[23:00:23] Warning: ambiguous stereochemistry - opposing bonds have opposite wedging - at atom 6 ignored.
100%|██████████| 16121/16121 [02:25<00:00, 111.14it/s]


_x_begin_max: 628.2038395818676, _x_end_min: 3798.05
------ ------ ------ ------ ------ ------
Preprocessing spectrum...


 28%|██▊       | 4470/16121 [00:40<01:31, 127.99it/s][23:02:23] Warning: ambiguous stereochemistry - opposing bonds have opposite wedging - at atom 6 ignored.
[23:02:23] Warning: ambiguous stereochemistry - opposing bonds have opposite wedging - at atom 6 ignored.
100%|██████████| 16121/16121 [02:06<00:00, 127.56it/s]


In [6]:
print(bad, "files couldn't be processed at all.")
from collections import Counter
print(Counter(reason_bad))

1 files couldn't be processed at all.
Counter({"Couldn't read file": 2384, 'LIQUID': 338, 'SOLID (KBr PELLET)': 312, 'SOLID (1 mg / 650 mg KBr DISC)': 292, 'SOLID (KBr DISC)': 280, 'SOLID (SPLIT MULL), FLUOROLUBE FOR 3800-1330 CM^-^1, NUJOL FOR 1330-400 CM^-^1': 257, 'SOLID (NUJOL MULL)': 200, 'SOLID (OIL MULL)': 194, 'SOLID (SPLIT MULL)': 188, 'SOLID (MINERAL OIL MULL)': 167, 'SOLUTION (10% CCl4 FOR 5000-1330, 10% CS2 FOR 1330-625 CM^-^1)': 155, 'LIQUID (NEAT)': 135, 'SOLID (0.8 mg / 650 mg KBr DISC)': 102, 'SOLID (1.0% IN KBr PELLET)': 92, 'SOLUTION (10% CCl4 FOR 3800-1330, 10% CS2 FOR 1330-400 CM^-^1)': 77, 'SOLID (KBr PRESSING)': 66, 'SOLID (SPLIT MULL), FLUOROLUBE FOR 3800-1330 CM^-^1, NUJOL FOR 1330-450 CM^-^1': 66, 'SOLID (SPLIT MULL), FLUOROLUBE FOR 3800-1330 CM^-^1, NUJOL FOR 1330-460 CM^-^1': 62, 'SOLUTION (10% IN CCl4 FOR 5000-1350 CM^-^1, 10% IN CS2 FOR 1350-625 CM^-^1)': 59, 'SOLUTION (10% CCl4 FOR 2.7-7.5, 10% CS2 FOR 7.5-26 MICRON) VS SOLVENT': 51, 'SOLID (0.9 mg / 650 m

In [7]:
datadict = {"SMILES": SMILES}
for i in range(spectra.shape[1]):
    datadict[i] = []
for spectrum in spectra:
    for i in range(spectra.shape[1]):
        datadict[i].append(spectrum[i])

dataset = pd.DataFrame(data = datadict)
dataset.to_csv("NIST Gaseous IR Dataset.csv")